In [1]:
from pathlib import Path
import zipfile
import json
from datasets import Dataset
from tqdm.auto import tqdm

In [2]:
BASE_DIR = "/mnt/d/data/한국어 음성"

In [3]:
# BASE_DIR 아래 모든 zip file 경로 수집
zip_files = list(Path(BASE_DIR).rglob("*.zip"))
print(f"Found {len(zip_files)} zip files.")
for zip_file in zip_files:
    print(zip_file)

Found 7 zip files.
/mnt/d/data/한국어 음성/평가용_데이터/KsponSpeech_eval.zip
/mnt/d/data/한국어 음성/한국어_음성_분야/KsponSpeech_02.zip
/mnt/d/data/한국어 음성/한국어_음성_분야/KsponSpeech_01.zip
/mnt/d/data/한국어 음성/한국어_음성_분야/KsponSpeech_03.zip
/mnt/d/data/한국어 음성/한국어_음성_분야/KsponSpeech_04.zip
/mnt/d/data/한국어 음성/한국어_음성_분야/KsponSpeech_05.zip
/mnt/d/data/한국어 음성/전시문_통합_스크립트/KsponSpeech_scripts.zip


In [4]:
# ZIP 파일 내용 확인
for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print(f"ZIP 파일 내부 파일 목록 ({zip_file}):")
        for file in file_list[:10]:  # 처음 10개만 출력
            print(file)

ZIP 파일 내부 파일 목록 (/mnt/d/data/한국어 음성/평가용_데이터/KsponSpeech_eval.zip):
eval_clean/
eval_clean/KsponSpeech_E00001.pcm
eval_clean/KsponSpeech_E00002.pcm
eval_clean/KsponSpeech_E00003.pcm
eval_clean/KsponSpeech_E00004.pcm
eval_clean/KsponSpeech_E00005.pcm
eval_clean/KsponSpeech_E00006.pcm
eval_clean/KsponSpeech_E00007.pcm
eval_clean/KsponSpeech_E00008.pcm
eval_clean/KsponSpeech_E00009.pcm
ZIP 파일 내부 파일 목록 (/mnt/d/data/한국어 음성/한국어_음성_분야/KsponSpeech_02.zip):
KsponSpeech_02/
KsponSpeech_02/KsponSpeech_0125/
KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124001.pcm
KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124001.txt
KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124002.pcm
KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124002.txt
KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124003.pcm
KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124003.txt
KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124004.pcm
KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124004.txt
ZIP 파일 내부 파일 목록 (/mnt/d/data/한국어 음성/한국어_음성_분야/KsponSpeech_0

In [14]:
# Install required packages for Whisper fine-tuning
!uv pip install transformers datasets accelerate evaluate jiwer tensorboard soundfile librosa

Using Python 3.10.18 environment at: /mnt/c/Users/LANDSOFT/Documents/dev/law/.venv
Audited 8 packages in 393ms


In [5]:
# # Extract audio files and transcripts from zip files
import os
import shutil

In [6]:
# Read audio and transcript files directly from ZIP files (no extraction)
import io
import re

# Separate audio zips and script zip
audio_zip_files = [z for z in zip_files if "scripts" not in str(z).lower() and "eval" not in str(z).lower()]
script_zip_file = [z for z in zip_files if "scripts" in str(z).lower()][0]

print(f"Audio ZIP files: {len(audio_zip_files)}")
print(f"Script ZIP file: {script_zip_file}")

# Load ALL transcripts from script ZIP
print("\nLoading ALL transcripts from script ZIP...")
all_transcripts = {}

with zipfile.ZipFile(script_zip_file, 'r') as zip_ref:
    transcript_files = [f for f in zip_ref.namelist() if f.endswith('.txt') or f.endswith('.trn')]
    print(f"Found {len(transcript_files)} transcript files in script ZIP")
    
    # Process ALL transcript files (not just first 10)
    for transcript_file in tqdm(transcript_files, desc="Parsing ALL transcripts"):
        with zip_ref.open(transcript_file) as f:
            content = f.read().decode('utf-8')
            for line in content.split('\n'):
                line = line.strip()
                if not line or '::' not in line:
                    continue
                parts = line.split(' :: ')
                if len(parts) == 2:
                    audio_path, transcript = parts
                    # Clean transcript but keep audio_path as-is (it's the key)
                    cleaned_transcript = re.sub(r'[/()\[\]<>]', '', transcript)
                    cleaned_transcript = re.sub(r'\s+', ' ', cleaned_transcript).strip()
                    # Use the FULL PATH (with .pcm) as the key
                    all_transcripts[audio_path.strip()] = cleaned_transcript

print(f"\nLoaded {len(all_transcripts)} transcripts")
print("Sample keys:", list(all_transcripts.keys())[:3])
print("Sample values:", list(all_transcripts.values())[:3])

Audio ZIP files: 5
Script ZIP file: /mnt/d/data/한국어 음성/전시문_통합_스크립트/KsponSpeech_scripts.zip

Loading ALL transcripts from script ZIP...
Found 4 transcript files in script ZIP


Parsing ALL transcripts:   0%|          | 0/4 [00:00<?, ?it/s]


Loaded 628545 transcripts
Sample keys: ['KsponSpeech_05/KsponSpeech_0621/KsponSpeech_620001.pcm', 'KsponSpeech_05/KsponSpeech_0621/KsponSpeech_620002.pcm', 'KsponSpeech_05/KsponSpeech_0621/KsponSpeech_620003.pcm']
Sample values: ['그러면 너랑 아 뭐 카페 가자는 거야?', 'o 두 분 다 근데 어디 출신이셔? 다 경상도?', '걔.']


In [7]:
# Collect audio files info from ZIP files (without extraction)
print("Scanning audio files in ZIP archives...")

audio_file_info = []  # List of (zip_path, file_path_in_zip, audio_id)

for zip_path in tqdm(audio_zip_files, desc="Scanning audio ZIPs"):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        pcm_files = [f for f in zip_ref.namelist() if f.endswith('.pcm')]
        for pcm_file in pcm_files:
            # Extract audio_id from filename
            audio_id = Path(pcm_file).stem
            if audio_id in all_transcripts:
                audio_file_info.append({
                    'zip_path': str(zip_path),
                    'file_in_zip': pcm_file,
                    'audio_id': audio_id,
                    'transcript': all_transcripts[audio_id]
                })

print(f"\nFound {len(audio_file_info)} matched audio-transcript pairs")
print("Sample:", audio_file_info[:2])

Scanning audio files in ZIP archives...


Scanning audio ZIPs:   0%|          | 0/5 [00:00<?, ?it/s]


Found 0 matched audio-transcript pairs
Sample: []


In [8]:
# FIXED: Collect audio files info with proper matching (use full path as key)
print("Scanning audio files in ZIP archives (matching with full path)...")

audio_file_info = []

for zip_path in tqdm(audio_zip_files, desc="Scanning audio ZIPs"):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        pcm_files = [f for f in zip_ref.namelist() if f.endswith('.pcm')]
        
        for pcm_file in pcm_files:
            # The transcript key is the full path WITH .pcm extension
            # So we use the pcm_file directly as the key
            if pcm_file in all_transcripts:
                audio_file_info.append({
                    'zip_path': str(zip_path),
                    'file_in_zip': pcm_file,
                    'audio_id': Path(pcm_file).stem,  # Use stem for display purposes
                    'transcript': all_transcripts[pcm_file]
                })

print(f"\nFound {len(audio_file_info)} matched audio-transcript pairs")
if audio_file_info:
    print("Sample matches:")
    for sample in audio_file_info[:3]:
        print(f"  Audio: {sample['file_in_zip']}")
        print(f"  ID: {sample['audio_id']}")
        print(f"  Transcript: {sample['transcript'][:80]}...")
        print()
else:
    print("\n⚠️  No matches found! Checking transcripts coverage...")
    print(f"Total transcripts loaded: {len(all_transcripts)}")
    print(f"Transcript files processed: Only first 10 (this is the issue!)")
    print("\n💡 Solution: Load ALL transcript files, not just first 10")

Scanning audio files in ZIP archives (matching with full path)...


Scanning audio ZIPs:   0%|          | 0/5 [00:00<?, ?it/s]


Found 622545 matched audio-transcript pairs
Sample matches:
  Audio: KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124001.pcm
  ID: KsponSpeech_124001
  Transcript: o 어. l...

  Audio: KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124002.pcm
  ID: KsponSpeech_124002
  Transcript: n 대명은 백퍼 있는데, b 오크밸리도 무룐지를 모르겠어. 그게 돈 내고 하는 거면, b 원주까지 내려가서 원주에서 무료 셔틀 타고 들어가는 게...

  Audio: KsponSpeech_02/KsponSpeech_0125/KsponSpeech_124003.pcm
  ID: KsponSpeech_124003
  Transcript: 이제 그 얘길 하면서 아 그럼 찾아보자. 해가지고 페이스북 계정을 급하게 만들어서. 그 여자애를 찾은 거야. 걔랑 b 그 다음에 b 내 딴 친구...



In [9]:
# Function to read PCM from ZIP and convert to numpy array
import numpy as np

def read_pcm_from_zip(zip_path: str, file_in_zip: str, sample_rate=16000) -> np.ndarray:
    """Read PCM file from ZIP and convert to numpy array"""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        with zip_ref.open(file_in_zip) as f:
            pcm_data = f.read()
            # Convert PCM bytes to numpy array (assuming 16-bit PCM)
            audio_array = np.frombuffer(pcm_data, dtype=np.int16).astype(np.float32)
            # Normalize to [-1, 1]
            audio_array = audio_array / 32768.0
            return audio_array

# Test reading one file
test_info = audio_file_info[0]
print("Testing PCM reading from ZIP...")
test_audio = read_pcm_from_zip(test_info['zip_path'], test_info['file_in_zip'])
print(f"Audio shape: {test_audio.shape}")
print(f"Audio duration: {len(test_audio) / 16000:.2f} seconds")
print(f"Transcript: {test_info['transcript']}")

Testing PCM reading from ZIP...
Audio shape: (37104,)
Audio duration: 2.32 seconds
Transcript: o 어. l


In [ ]:
from datasets import Dataset, DatasetDict, Features, Value, Audio
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import cpu_count
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import gc
import os

def load_single_sample(info):
    """Load a single audio-transcript pair"""
    try:
        import zipfile
        import numpy as np
        
        with zipfile.ZipFile(info['zip_path'], 'r') as zip_ref:
            with zip_ref.open(info['file_in_zip']) as f:
                pcm_data = f.read()
                audio_array = np.frombuffer(pcm_data, dtype=np.int16).astype(np.float32)
                audio_array = audio_array / 32768.0
        
        result = {
            'audio': {
                'array': audio_array,
                'sampling_rate': 16000
            },
            'transcription': info['transcript'],
            'audio_id': info['audio_id']
        }
        
        del pcm_data
        gc.collect()
        
        return result
        
    except Exception as e:
        print(f"Error loading {info['audio_id']}: {e}")
        return None

def generate_dataset_streaming(audio_file_info, max_samples=None, max_workers=None, 
                               chunk_size=100):
    """FFmpeg 불필요한 generator"""
    samples = audio_file_info[:max_samples] if max_samples else audio_file_info
    
    if max_workers is None:
        max_workers = cpu_count()
    
    print(f"Processing {len(samples)} samples with {max_workers} workers")
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        processed = 0
        for i in range(0, len(samples), chunk_size):
            chunk = samples[i:i + chunk_size]
            futures = [executor.submit(load_single_sample, info) for info in chunk]
            
            for future in as_completed(futures):
                result = future.result()
                if result is not None:
                    yield result
                    processed += 1
                    
                    if processed % 100 == 0:
                        print(f"Processed {processed}/{len(samples)} samples")
            
            gc.collect()

# ===== Features 정의 (Audio 타입 사용) =====
features = Features({
    'audio': Audio(sampling_rate=16000),
    'transcription': Value('string'),
    'audio_id': Value('string')
})

# ===== 캐시 디렉토리 생성 =====
cache_dir = "/mnt/e/tmp/dataset_cache"
os.makedirs(cache_dir, exist_ok=True)

# ===== 데이터셋 생성 =====
print("Creating dataset from generator...")
dataset = Dataset.from_generator(
    lambda: generate_dataset_streaming(audio_file_info),
    features=features,
    cache_dir=cache_dir
)

print(f"✅ Dataset created: {len(dataset)} samples")

Creating dataset from generator...


Generating train split: 0 examples [00:00, ? examples/s]

Processing 622545 samples with 16 workers
Processed 100/622545 samples
Processed 200/622545 samples
Processed 300/622545 samples
Processed 400/622545 samples
Processed 500/622545 samples
Processed 600/622545 samples
Processed 700/622545 samples
Processed 800/622545 samples
Processed 900/622545 samples
Processed 1000/622545 samples
Processed 1100/622545 samples
Processed 1200/622545 samples
Processed 1300/622545 samples
Processed 1400/622545 samples
Processed 1500/622545 samples
Processed 1600/622545 samples
Processed 1700/622545 samples
Processed 1800/622545 samples
Processed 1900/622545 samples
Processed 2000/622545 samples
Processed 2100/622545 samples
Processed 2200/622545 samples
Processed 2300/622545 samples
Processed 2400/622545 samples
Processed 2500/622545 samples
Processed 2600/622545 samples
Processed 2700/622545 samples
Processed 2800/622545 samples
Processed 2900/622545 samples
Processed 3000/622545 samples
Processed 3100/622545 samples
Processed 3200/622545 samples
Process

Loading dataset shards:   0%|          | 0/220 [00:00<?, ?it/s]

✅ Dataset created: 622545 samples


ImportError: To support decoding audio data, please install 'torchcodec'.

In [15]:
# print(f"Sample keys: {dataset[0].keys()}")
# print(f"Audio array shape: {dataset[0]['audio']['array'].shape}")
# print(f"Sampling rate: {dataset[0]['audio']['sampling_rate']}")
# print(f"Transcription: {dataset[0]['transcription'][:80]}...")

# ===== Hub에 업로드 =====


print("\nPushing to Hub...")
dataset.push_to_hub(
    "aihub-korean-whisper-full-streaming",
    private=False,
    max_shard_size="500MB"
)

print("✅ Upload complete!")


Pushing to Hub...


Uploading the dataset shards:   0%|          | 0/224 [00:00<?, ? shards/s]

Map:   0%|          | 0/2780 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/401 [00:00<?, ?B/s]

✅ Upload complete!


In [16]:
# Split dataset into train/validation
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(f"Train dataset: {len(train_dataset)} samples")
print(f"Eval dataset: {len(eval_dataset)} samples")

Train dataset: 560290 samples
Eval dataset: 62255 samples


In [16]:
# Load Whisper model and processor (with CUDA safety checks)
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import gc

# Clear CUDA cache first
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("CUDA cache cleared")

model_name = "openai/whisper-small"  # Can use tiny, base, small, medium, large

print(f"Loading {model_name}...")
processor = WhisperProcessor.from_pretrained(model_name, language="korean", task="transcribe")

# Load model and move to GPU
model = WhisperForConditionalGeneration.from_pretrained(model_name)

if torch.cuda.is_available():
    model = model.to("cuda")
    print(f"Model moved to GPU: {torch.cuda.get_device_name(0)}")

# Force Korean language tokens
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="korean", task="transcribe")
model.config.suppress_tokens = []

print(f"\n✅ Loaded model: {model_name}")
print(f"Model parameters: {model.num_parameters():,}")

if torch.cuda.is_available():
    print(f"GPU Memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU Memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

CUDA cache cleared
Loading openai/whisper-small...
Model moved to GPU: NVIDIA GeForce RTX 2060 SUPER

✅ Loaded model: openai/whisper-small
Model parameters: 241,734,912
GPU Memory allocated: 0.90 GB
GPU Memory cached: 0.99 GB


In [ ]:
# Check GPU availability and memory
import torch

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"Cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
else:
    print("⚠️ No GPU available! Training will be very slow on CPU.")
    print("Consider using Google Colab or a machine with GPU.")

GPU: NVIDIA GeForce RTX 2060 SUPER
CUDA Version: 11.8
Total GPU Memory: 8.00 GB
Allocated: 0.90 GB
Cached: 0.99 GB


In [ ]:
# Prepare dataset for training (with label validation)
from transformers import WhisperFeatureExtractor, WhisperTokenizer
from dataclasses import dataclass
from typing import Any, Dict, List, Union

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="korean", task="transcribe")

def prepare_dataset(batch):
    """Prepare audio and text for Whisper training with label validation"""
    # Load and process audio
    audio = batch["audio"]
    
    # Compute log-Mel input features
    batch["input_features"] = feature_extractor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    
    # Encode target text to label ids
    encoded = tokenizer(batch["transcription"])
    labels = encoded.input_ids
    
    # CRITICAL: Remove any token IDs that exceed vocabulary size
    vocab_size = tokenizer.vocab_size
    labels = [token_id for token_id in labels if token_id < vocab_size]
    
    # If all tokens were invalid, use empty sequence (will be handled by loss masking)
    if not labels:
        labels = [tokenizer.eos_token_id]
    
    batch["labels"] = labels
    
    return batch

# Apply preprocessing
print("Preprocessing train dataset with label validation...")
train_dataset = train_dataset.map(
    prepare_dataset, 
    remove_columns=train_dataset.column_names,
    num_proc=4
)

print("Preprocessing eval dataset with label validation...")
eval_dataset = eval_dataset.map(
    prepare_dataset, 
    remove_columns=eval_dataset.column_names,
    num_proc=4
)

print("✅ Dataset preprocessing complete with label validation!")

Preprocessing train dataset with label validation...


Map (num_proc=4): 100%|██████████| 900/900 [00:24<00:00, 37.18 examples/s]


Preprocessing eval dataset with label validation...


Map (num_proc=4): 100%|██████████| 100/100 [00:13<00:00,  7.34 examples/s]

✅ Dataset preprocessing complete with label validation!


In [ ]:
# from datasets import DatasetDict

# ds = DatasetDict({
#     "train": train_dataset,
#     "eval": eval_dataset
# })
# ds

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 900
    })
    eval: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 100
    })
})

In [ ]:
# ds.push_to_hub("aihub-korean-whisper-small-dataset")

Creating parquet from Arrow format: 100%|██████████| 5/5 [00:01<00:00,  3.11ba/s]
Processing Files (1 / 1): 100%|██████████| 85.4MB / 85.4MB, 7.89MB/s  
New Data Upload: 100%|██████████| 85.4MB / 85.4MB, 7.89MB/s  
Creating parquet from Arrow format: 100%|██████████| 5/5 [00:01<00:00,  3.02ba/s]
Processing Files (1 / 1): 100%|██████████| 88.1MB / 88.1MB, 7.45MB/s  
New Data Upload: 100%|██████████| 88.1MB / 88.1MB, 7.45MB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  3.25ba/s]
Processing Files (1 / 1): 100%|██████████| 20.1MB / 20.1MB, 5.03MB/s  
New Data Upload: 100%|██████████| 20.1MB / 20.1MB, 5.03MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:05<00:00,  5.34s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/brainer/aihub-korean-whisper-small-dataset/commit/6bb0ef8b2b054f7ace9ee65569c37c512614e66e', commit_message='Upload dataset', commit_description='', oid='6bb0ef8b2b054f7ace9ee65569c37c512614e66e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/brainer/aihub-korean-whisper-small-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='brainer/aihub-korean-whisper-small-dataset'), pr_revision=None, pr_num=None)

In [19]:
# Check tokenizer vocabulary size and label values
print(f"Tokenizer vocabulary size: {tokenizer.vocab_size}")
print(f"Model vocabulary size: {model.config.vocab_size}")

# Check a few samples from processed dataset
print("\nChecking label ranges in processed dataset...")
max_label_values = []
for i in range(min(10, len(train_dataset))):
    labels = train_dataset[i]['labels']
    max_val = max(labels)
    max_label_values.append(max_val)
    if max_val >= tokenizer.vocab_size:
        print(f"❌ Sample {i}: max label = {max_val} (exceeds vocab size {tokenizer.vocab_size})")
    else:
        print(f"✅ Sample {i}: max label = {max_val}")

print(f"\nOverall max label value: {max(max_label_values)}")
print(f"Vocab size: {tokenizer.vocab_size}")

Tokenizer vocabulary size: 50258
Model vocabulary size: 51865

Checking label ranges in processed dataset...
✅ Sample 0: max label = 50257
✅ Sample 1: max label = 50257
✅ Sample 2: max label = 50257
✅ Sample 3: max label = 50257
✅ Sample 4: max label = 50257
✅ Sample 5: max label = 50257
✅ Sample 6: max label = 50257
✅ Sample 7: max label = 50257
✅ Sample 8: max label = 50257
✅ Sample 9: max label = 50257

Overall max label value: 50257
Vocab size: 50258


## Whisper Fine-tuning Summary

### Pipeline Overview
1. **Data Extraction**: Extract audio and transcript files from KsponSpeech zip files
2. **Format Conversion**: Convert PCM audio to WAV format for Whisper compatibility
3. **Dataset Creation**: Match audio files with transcripts and create HuggingFace Dataset
4. **Preprocessing**: Convert audio to log-Mel spectrograms and tokenize transcripts
5. **Training**: Fine-tune Whisper model with Korean speech data
6. **Evaluation**: Monitor WER (Word Error Rate) during training

### Key Configuration
- **Model**: openai/whisper-small (can upgrade to medium/large)
- **Language**: Korean
- **Batch Size**: 16 (8 per device × 2 accumulation steps)
- **Learning Rate**: 1e-5
- **Training Steps**: 5,000
- **Evaluation Metric**: WER (Word Error Rate)

### Training Tips
- Adjust `per_device_train_batch_size` based on available GPU memory
- Use `gradient_checkpointing=True` to reduce memory usage
- Monitor tensorboard logs: `tensorboard --logdir /mnt/d/data/한국어 음성/whisper-korean-finetuned`
- Best model is automatically saved based on lowest WER

### Next Steps
- Increase sample size beyond 1,000 files for better performance
- Process all zip files for full dataset
- Experiment with different Whisper model sizes
- Fine-tune hyperparameters based on validation results

## Memory-Efficient Dataset Creation with Lazy Loading

To avoid memory overflow when processing the entire dataset, we'll use a generator-based approach that:
1. Processes data in batches
2. Streams directly to HuggingFace Hub
3. Never loads the entire dataset into memory

In [12]:
# Generator function to yield samples lazily (memory-efficient)
def audio_sample_generator(audio_file_info, batch_size=100):
    """
    Lazy generator that yields audio samples in batches
    This avoids loading all data into memory at once
    """
    import zipfile
    import numpy as np
    from tqdm.auto import tqdm
    
    total_samples = len(audio_file_info)
    
    for i in tqdm(range(0, total_samples, batch_size), desc="Processing batches"):
        batch_info = audio_file_info[i:i+batch_size]
        
        for info in batch_info:
            try:
                # Read PCM from ZIP
                with zipfile.ZipFile(info['zip_path'], 'r') as zip_ref:
                    with zip_ref.open(info['file_in_zip']) as f:
                        pcm_data = f.read()
                        audio_array = np.frombuffer(pcm_data, dtype=np.int16).astype(np.float32)
                        audio_array = audio_array / 32768.0
                
                yield {
                    'audio': {
                        'array': audio_array,
                        'sampling_rate': 16000
                    },
                    'transcription': info['transcript'],
                    'audio_id': info['audio_id']
                }
            except Exception as e:
                print(f"Error loading {info['audio_id']}: {e}")
                continue

print(f"Total audio files to process: {len(audio_file_info)}")
print("Generator function ready - data will be loaded lazily during iteration")

Total audio files to process: 622545
Generator function ready - data will be loaded lazily during iteration


In [20]:
# Create dataset from generator (lazy loading)
from datasets import Dataset, IterableDataset, Features, Value, Audio, Sequence

# Define features explicitly for IterableDataset
features = Features({
    'audio': Audio(sampling_rate=16000),
    'transcription': Value('string'),
    'audio_id': Value('string')
})

# Use IterableDataset for lazy loading with explicit features
iterable_dataset = IterableDataset.from_generator(
    audio_sample_generator,
    gen_kwargs={'audio_file_info': audio_file_info, 'batch_size': 100},
    features=features
)

print("✅ Iterable dataset created (data not loaded yet)")
print("Data will be loaded on-demand during iteration or push_to_hub")

✅ Iterable dataset created (data not loaded yet)
Data will be loaded on-demand during iteration or push_to_hub


In [14]:
# Test the generator by taking a few samples
print("Testing generator with first 3 samples...")
for i, sample in enumerate(iterable_dataset.take(3)):
    print(f"\nSample {i+1}:")
    print(f"  Audio ID: {sample['audio_id']}")
    print(f"  Audio length: {len(sample['audio']['array'])} samples")
    print(f"  Transcription: {sample['transcription'][:80]}...")
    print(f"  Duration: {len(sample['audio']['array']) / 16000:.2f} seconds")

Testing generator with first 3 samples...


Processing batches: 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]



Sample 1:
  Audio ID: KsponSpeech_124001
  Audio length: 37104 samples
  Transcription: o 어. l...
  Duration: 2.32 seconds


Processing batches: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]



Sample 2:
  Audio ID: KsponSpeech_124002
  Audio length: 176288 samples
  Transcription: n 대명은 백퍼 있는데, b 오크밸리도 무룐지를 모르겠어. 그게 돈 내고 하는 거면, b 원주까지 내려가서 원주에서 무료 셔틀 타고 들어가는 게...
  Duration: 11.02 seconds


Processing batches:   0%|          | 0/1 [00:01<?, ?it/s]


Sample 3:
  Audio ID: KsponSpeech_124003
  Audio length: 238848 samples
  Transcription: 이제 그 얘길 하면서 아 그럼 찾아보자. 해가지고 페이스북 계정을 급하게 만들어서. 그 여자애를 찾은 거야. 걔랑 b 그 다음에 b 내 딴 친구...
  Duration: 14.93 seconds


In [21]:
# Preprocessing function for iterable dataset
from transformers import WhisperFeatureExtractor, WhisperTokenizer

# Ensure feature extractor and tokenizer are initialized
if 'feature_extractor' not in locals():
    feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
if 'tokenizer' not in locals():
    tokenizer = WhisperTokenizer.from_pretrained(model_name, language="korean", task="transcribe")

def prepare_dataset_streaming(batch):
    """Prepare audio and text for Whisper training (streaming version)"""
    # Compute log-Mel input features
    batch["input_features"] = feature_extractor(
        batch["audio"]["array"], 
        sampling_rate=batch["audio"]["sampling_rate"]
    ).input_features[0]
    
    # Encode target text to label ids
    encoded = tokenizer(batch["transcription"])
    labels = encoded.input_ids
    
    # Remove any token IDs that exceed vocabulary size
    vocab_size = tokenizer.vocab_size
    labels = [token_id for token_id in labels if token_id < vocab_size]
    
    # If all tokens were invalid, use empty sequence
    if not labels:
        labels = [tokenizer.eos_token_id]
    
    batch["labels"] = labels
    
    # Remove audio to save memory after processing
    del batch["audio"]
    
    return batch

# Apply preprocessing to iterable dataset
print("Setting up preprocessing pipeline (lazy - will process during iteration)...")
processed_dataset = iterable_dataset.map(
    prepare_dataset_streaming,
    remove_columns=['audio', 'audio_id', 'transcription']
)

print("✅ Preprocessing pipeline configured")
print("Data will be preprocessed on-the-fly during push_to_hub")

Setting up preprocessing pipeline (lazy - will process during iteration)...
✅ Preprocessing pipeline configured
Data will be preprocessed on-the-fly during push_to_hub


In [22]:
# Push to HuggingFace Hub with streaming (memory-efficient)
# This will process and upload data in batches without loading everything into memory

dataset_name = "aihub-korean-whisper-full-streaming"

print(f"Starting streaming upload to HuggingFace Hub: {dataset_name}")
print(f"Total files to process: {len(audio_file_info)}")
print("\n⚠️  This will take a while as it processes and uploads data in batches")
print("Progress will be shown during upload...\n")

# Push the iterable dataset to hub
# The data will be processed and uploaded in chunks
# Note: IterableDataset doesn't support max_shard_size parameter
processed_dataset.push_to_hub(
    dataset_name,
    private=False  # Set to True if you want private dataset
)

print(f"\n✅ Dataset successfully uploaded to: {dataset_name}")
print("You can now load it with: datasets.load_dataset('brainer/{dataset_name}')")

Starting streaming upload to HuggingFace Hub: aihub-korean-whisper-full-streaming
Total files to process: 622545

⚠️  This will take a while as it processes and uploads data in batches
Progress will be shown during upload...



AttributeError: 'NoneType' object has no attribute 'items'

## Alternative: Split Dataset for Train/Eval Before Uploading

If you want to split into train/eval sets before uploading, you can create separate generators:

In [ ]:
# Optional: Create train/eval split before uploading
import random

# Shuffle and split indices
random.seed(42)
indices = list(range(len(audio_file_info)))
random.shuffle(indices)

split_point = int(len(indices) * 0.9)  # 90% train, 10% eval
train_indices = set(indices[:split_point])
eval_indices = set(indices[split_point:])

print(f"Train samples: {len(train_indices)}")
print(f"Eval samples: {len(eval_indices)}")

def audio_sample_generator_split(audio_file_info, indices_set, batch_size=100):
    """Generator for specific indices (train or eval)"""
    import zipfile
    import numpy as np
    from tqdm.auto import tqdm
    
    filtered_info = [info for i, info in enumerate(audio_file_info) if i in indices_set]
    
    for i in tqdm(range(0, len(filtered_info), batch_size), desc="Processing batches"):
        batch_info = filtered_info[i:i+batch_size]
        
        for info in batch_info:
            try:
                with zipfile.ZipFile(info['zip_path'], 'r') as zip_ref:
                    with zip_ref.open(info['file_in_zip']) as f:
                        pcm_data = f.read()
                        audio_array = np.frombuffer(pcm_data, dtype=np.int16).astype(np.float32)
                        audio_array = audio_array / 32768.0
                
                yield {
                    'audio': {
                        'array': audio_array,
                        'sampling_rate': 16000
                    },
                    'transcription': info['transcript'],
                    'audio_id': info['audio_id']
                }
            except Exception as e:
                print(f"Error loading {info['audio_id']}: {e}")
                continue

# Define features for train/eval datasets
from datasets import Features, Value, Audio

features_split = Features({
    'audio': Audio(sampling_rate=16000),
    'transcription': Value('string'),
    'audio_id': Value('string')
})

# Create separate iterable datasets for train and eval with explicit features
train_iterable = IterableDataset.from_generator(
    audio_sample_generator_split,
    gen_kwargs={'audio_file_info': audio_file_info, 'indices_set': train_indices, 'batch_size': 100},
    features=features_split
)

eval_iterable = IterableDataset.from_generator(
    audio_sample_generator_split,
    gen_kwargs={'audio_file_info': audio_file_info, 'indices_set': eval_indices, 'batch_size': 100},
    features=features_split
)

# Apply preprocessing
train_processed = train_iterable.map(prepare_dataset_streaming, remove_columns=['audio', 'audio_id', 'transcription'])
eval_processed = eval_iterable.map(prepare_dataset_streaming, remove_columns=['audio', 'audio_id', 'transcription'])

print("\n✅ Train and eval datasets created with lazy loading")

In [ ]:
# Push train and eval datasets separately to HuggingFace Hub
from datasets import DatasetDict

# Create a DatasetDict with train and eval splits
dataset_dict_splits = DatasetDict({
    "train": train_processed,
    "eval": eval_processed
})

dataset_name_with_splits = "aihub-korean-whisper-full-splits"

print(f"Starting streaming upload with train/eval splits to: {dataset_name_with_splits}")
print(f"Train samples: {len(train_indices)}")
print(f"Eval samples: {len(eval_indices)}")
print("\n⚠️  This will process and upload data in batches (may take hours for large dataset)")
print("Progress will be shown during upload...\n")

# Push to hub with splits
# Note: IterableDataset doesn't support max_shard_size parameter
dataset_dict_splits.push_to_hub(
    dataset_name_with_splits,
    private=False
)

print(f"\n✅ Dataset with splits successfully uploaded to: {dataset_name_with_splits}")
print("Load with: datasets.load_dataset('your-username/{dataset_name_with_splits}')")

## Summary: Memory-Efficient Dataset Pipeline

### Key Changes for Memory Efficiency:

1. **Generator-based loading**: Uses Python generators to yield samples one at a time instead of loading all data into memory
2. **IterableDataset**: HuggingFace's streaming dataset type that processes data on-the-fly
3. **Lazy preprocessing**: Features and labels are computed during iteration, not upfront
4. **Batched upload**: `push_to_hub` uploads data in shards (500MB each) without loading the entire dataset
5. **Memory cleanup**: Audio arrays are deleted after feature extraction

### Advantages:

- **No memory overflow**: Can process datasets larger than available RAM
- **Immediate start**: No waiting for full dataset to load before upload begins
- **Fault tolerance**: Sharded upload means partial progress is saved
- **Scalability**: Works with datasets of any size (100GB+)

### Usage:

```python
# After upload completes, load the dataset for training:
from datasets import load_dataset

dataset = load_dataset("your-username/aihub-korean-whisper-full-splits")
train_data = dataset["train"]
eval_data = dataset["eval"]

# The dataset will still stream efficiently during training
```

### Estimated Processing Time:

- **~1,000 samples**: 5-10 minutes
- **~10,000 samples**: 30-60 minutes  
- **~100,000 samples**: 5-10 hours
- **Full dataset (500K+)**: 24-48 hours

Time depends on: disk I/O speed, network bandwidth, CPU cores, preprocessing complexity.